In [84]:
import requests
import requests
import urllib
import pprint
import re
from bs4 import BeautifulSoup

class getFreeProxies():
    def __init__(self, url= "https://free-proxy-list.net/uk-proxy.html"):
        self.FREE_PROXIES = url
        self.raw = get_raw_from_url(FREE_PROXIES)
        self.proxies = get_useful_proxies(raw)
        pprint.pprint(proxies)
    

    def get_raw_from_url(self, url):
        return requests.get(url).text

    def scrape_website(self, code):
        soup = BeautifulSoup(code, 'html.parser')
        proxy_entry = soup.find_all("tr")#, {"class":"odd"})
        _proxies = []
        for row in proxy_entry:
            single = row.find_all("td")
            if(len(single) == 8):

                if(single[4].text == "anonymous"):
                    # return format: dictionary
                    # 'http(s):'ip:port''
                    # to use it as proxies dict 
                    # with request later on
                    key = 'https' if (single[6].text == "yes") else 'http'
                    value = single[0].text.strip()+":"+single[1].text.strip()
                    _proxies.append({key:value})
        return _proxies
    
    def get_proxies(self):
        return self.proxies

a = getFreeProxies()
a.get_proxies()


[{'http': '139.59.169.246:8080'},
 {'http': '138.68.161.60:8080'},
 {'http': '138.68.161.14:3128'},
 {'http': '138.68.173.29:8080'},
 {'http': '138.68.165.154:3128'},
 {'https': '54.37.18.25:8080'},
 {'http': '5.252.161.48:8080'},
 {'https': '134.209.179.96:3128'},
 {'http': '154.16.63.16:8080'},
 {'http': '185.134.23.172:80'},
 {'http': '185.122.58.80:80'},
 {'http': '185.134.23.173:80'},
 {'http': '185.186.77.244:80'},
 {'http': '185.134.23.171:80'}]


[{'http': '139.59.169.246:8080'},
 {'http': '138.68.161.60:8080'},
 {'http': '138.68.161.14:3128'},
 {'http': '138.68.173.29:8080'},
 {'http': '138.68.165.154:3128'},
 {'https': '54.37.18.25:8080'},
 {'http': '5.252.161.48:8080'},
 {'https': '134.209.179.96:3128'},
 {'http': '154.16.63.16:8080'},
 {'http': '185.134.23.172:80'},
 {'http': '185.122.58.80:80'},
 {'http': '185.134.23.173:80'},
 {'http': '185.186.77.244:80'},
 {'http': '185.134.23.171:80'}]